In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

### We will be working on main dataset in this notebook

In [2]:
# merging the 10-part sales data
main = {} # for storing all the 10 main datasets
for i in range(1,11):
    main[i] = pd.read_excel(open('C:/Users/Sword Guo/OneDrive/Practicum/Datasets/Labelmaster Daily Sales by Product Group Part ' + str(i) + '.xlsx', 'rb'))
    main[i].columns = main[i].iloc[1, :]
    main[i] = main[i].drop(index=[0, 1], axis=0)

l = [main[i] for i in range(1,11)]
main_data = pd.concat(l)
main_data

1,Department Dim,Product Group Code,Posting Date,Sum of Sales
2,503,50304,1993-01-04 00:00:00,76
3,505,50500,1993-01-04 00:00:00,5434.7
4,505,50502,1993-01-04 00:00:00,2785
5,506,50600,1993-01-04 00:00:00,129.76
6,506,51008,1993-01-04 00:00:00,43
...,...,...,...,...
2610,512,51211,2020-10-31 00:00:00,183.27
2611,514,51401,2020-10-31 00:00:00,0
2612,514,51402,2020-10-31 00:00:00,114
2613,516,51601,2020-10-31 00:00:00,19676.42


#### We only need Books and Packaging sales data. 

In [3]:
books = main_data[main_data['Department Dim'] == '504']
books

1,Department Dim,Product Group Code,Posting Date,Sum of Sales
67,504,NaN,1993-01-19 00:00:00,7894.7
127,504,NaN,1993-02-05 00:00:00,22344.2
149,504,NaN,1993-02-11 00:00:00,29866.15
201,504,NaN,1993-02-25 00:00:00,67568.81
297,504,NaN,1993-03-23 00:00:00,21483.3
...,...,...,...,...
2542,504,50407,2020-10-30 00:00:00,0
2543,504,50408,2020-10-30 00:00:00,0
2544,504,50409,2020-10-30 00:00:00,504
2594,504,50400,2020-10-31 00:00:00,59.78


In [4]:
books.sort_values(by='Posting Date', inplace=True) # sorting the data based on date
books.reset_index(drop=True, inplace=True)
books['Posting Date'] = [str(i) for i in books['Posting Date']]

# We will be needing month and year for merging with the external data
books[['Date', 'Time']] = books['Posting Date'].str.split(' ', expand=True)

y_m = []
for i in books['Date']:
    y_m.append(''.join(re.findall('\d+-\d+', i)))
    
books['Year_Month'] = pd.Series(y_m)
books

1,Department Dim,Product Group Code,Posting Date,Sum of Sales,Date,Time,Year_Month
0,504,NaN,1993-01-04 00:00:00,7963.55,1993-01-04,00:00:00,1993-01
1,504,NaN,1993-01-05 00:00:00,14667.7,1993-01-05,00:00:00,1993-01
2,504,NaN,1993-01-06 00:00:00,8421.5,1993-01-06,00:00:00,1993-01
3,504,NaN,1993-01-07 00:00:00,11616.85,1993-01-07,00:00:00,1993-01
4,504,NaN,1993-01-08 00:00:00,15092.72,1993-01-08,00:00:00,1993-01
...,...,...,...,...,...,...,...
160181,504,50404,2021-01-07 00:00:00,0,2021-01-07,00:00:00,2021-01
160182,504,50406,2021-01-07 00:00:00,0,2021-01-07,00:00:00,2021-01
160183,504,50407,2021-01-07 00:00:00,0,2021-01-07,00:00:00,2021-01
160184,504,50408,2021-01-07 00:00:00,146,2021-01-07,00:00:00,2021-01


In [5]:
# saving books dataset for future use.
# books.to_csv('Books_data.csv')

In [77]:
books_agg = books.groupby('Year_Month').agg({'Sum of Sales':'sum'})
books_agg = books_agg.reset_index(drop= False)
books_agg

,Year_Month,Sum of Sales
0,1993-01,236875.51
1,1993-02,389034.46
2,1993-03,221486.85
3,1993-04,331739.09
4,1993-05,325079.50
...,...,...
329,2020-09,468710.96
330,2020-10,1118446.80
331,2020-11,429166.77
332,2020-12,549415.01


In [6]:
# Doing the same for packaging
packaging = main_data[main_data['Department Dim'] == '506']
packaging

1,Department Dim,Product Group Code,Posting Date,Sum of Sales
5,506,50600,1993-01-04 00:00:00,129.76
6,506,51008,1993-01-04 00:00:00,43
10,506,NaN,1993-01-05 00:00:00,5904.97
15,506,50607,1993-01-06 00:00:00,155.4
16,506,50608,1993-01-06 00:00:00,37.08
...,...,...,...,...
2561,506,50616,2020-10-30 00:00:00,16.58
2598,506,50600,2020-10-31 00:00:00,-8.9
2599,506,50602,2020-10-31 00:00:00,197.7
2600,506,50605,2020-10-31 00:00:00,424.08


In [7]:
packaging.sort_values(by='Posting Date', inplace=True)
packaging.reset_index(drop=True, inplace=True)
packaging['Posting Date'] = [str(i) for i in packaging['Posting Date']]
packaging[['Date', 'Time']] = packaging['Posting Date'].str.split(' ', expand=True)

y_m = []
for i in packaging['Date']:
    y_m.append(''.join(re.findall('\d+-\d+', i)))

packaging['Year_Month'] = pd.Series(y_m)
packaging

1,Department Dim,Product Group Code,Posting Date,Sum of Sales,Date,Time,Year_Month
0,506,50600,1993-01-04 00:00:00,129.76,1993-01-04,00:00:00,1993-01
1,506,50613,1993-01-04 00:00:00,30.9,1993-01-04,00:00:00,1993-01
2,506,50604,1993-01-04 00:00:00,1117.2,1993-01-04,00:00:00,1993-01
3,506,50606,1993-01-04 00:00:00,361,1993-01-04,00:00:00,1993-01
4,506,NaN,1993-01-04 00:00:00,3195.52,1993-01-04,00:00:00,1993-01
...,...,...,...,...,...,...,...
189010,506,50609,2021-01-07 00:00:00,72.2,2021-01-07,00:00:00,2021-01
189011,506,50616,2021-01-07 00:00:00,82.01,2021-01-07,00:00:00,2021-01
189012,506,50613,2021-01-07 00:00:00,712.92,2021-01-07,00:00:00,2021-01
189013,506,50614,2021-01-07 00:00:00,266,2021-01-07,00:00:00,2021-01


In [8]:
# Saving packagin data for future use
# packaging.to_csv('Packaging_data.csv')

In [73]:
packaging_agg = packaging.groupby('Year_Month').agg({'Sum of Sales': 'sum'})
packaging_agg = packaging_agg.reset_index(drop= False)
packaging_agg

,Year_Month,Sum of Sales
0,1993-01,263979.80
1,1993-02,224439.85
2,1993-03,245191.23
3,1993-04,244614.47
4,1993-05,212929.28
...,...,...
329,2020-09,980476.36
330,2020-10,922901.36
331,2020-11,293486.93
332,2020-12,557010.14


### Only keep the data from 2008- 2020 from external dataset

In [59]:
external = pd.read_csv("C:/Users/Sword Guo/OneDrive/Practicum/LabelMaster_Sales_Forecasting/data/external_monthly_data.csv", index_col=0)
external = external.drop(external.index[344:347])
external

,"FTR Truck Loadings (000s, SA)_freight_m_trucking",FTR Truck Loadings Index (2000=100)_freight_m_trucking,"FTR Active Truck Utilization (%, SA)_freight_m_trucking","Dry Van Trailer Loadings (000s, SA)_freight_m_trucking","Reefer Trailer Loadings (000s, SA)_freight_m_trucking","Flatbed Trailer Loadings (000s, SA)_freight_m_trucking","Tank Trailer Loadings (000s, SA)_freight_m_trucking","Bulk Trailer/Straight Truck Loadings (000s, SA)_freight_m_trucking","Food & Kindred Products (000s, SA)_freight_m_trucking","Stone, Clay, Glass & Concrete (000s, SA)_freight_m_trucking",...,"East Coast Port Activity, Total TEUs_Exports_y.1","East Coast Port Activity, Total TEUs_SA Imports_y.1","Gulf Coast Port Activity, Total TEUs_Imports_y.1","Gulf Coast Port Activity, Total TEUs_Exports_y.1","Gulf Coast Port Activity, Total TEUs_SA Imports_y.1","Western Canadian Port Activity, Total TEUs_Imports_y.1","Western Canadian Port Activity, Total TEUs_Exports_y.1","Western Canadian Port Activity, Total TEUs_SA Imports_y.1",Total Intermodal (w/o FSC)_y.1,Total Intermodal (w/ FSC)_y.1
0,44421.579068,79.139972,83.420869,15186.045111,2760.592455,4739.934294,5779.736522,15955.270687,6620.956039,3897.838060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44679.069698,79.598709,83.692980,15280.932319,2782.330324,4762.928519,5741.757514,16111.121021,6604.793809,3899.166170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,44218.326435,78.777864,83.704951,15314.637594,2799.054591,4747.635503,5723.672462,15633.326285,6620.591404,3895.224827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,44203.801340,78.751987,85.107661,15288.281309,2836.712781,4714.178945,5498.191034,15866.437273,6623.565120,3867.088439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44111.883737,78.588230,84.688849,15263.423620,2818.627081,4700.889440,5449.097577,15879.846019,6614.689544,3862.912776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,61469.059874,109.511184,89.962431,20072.391023,4379.187334,6993.332683,6571.127779,23453.021054,9375.896057,5404.795100,...,472910.156297,8.175415e+05,131560.759686,125276.157772,124175.287958,225515.00000,103172.000000,203343.596376,119.865470,128.935612
340,62140.797176,110.707928,90.641835,20379.910791,4404.415547,7096.862705,6601.260917,23658.347216,9520.297567,5381.920841,...,499597.784894,9.441444e+05,150026.806042,125319.157772,144616.508568,235159.00000,93979.000000,212554.287841,121.915166,130.511416
341,62170.174317,110.760265,90.587889,20315.722086,4381.188114,7049.924576,6556.745680,23866.593861,9472.590699,5253.495206,...,492325.048120,9.480849e+05,156153.318284,117576.128306,154165.434806,216790.00000,103129.000000,200875.373073,123.509489,131.952815
342,62870.759335,112.008404,94.367218,20404.155544,4436.920215,7199.060314,6523.796564,24306.826698,9565.479621,5479.173341,...,546879.252290,9.980584e+05,173631.303295,123604.184721,162787.138608,260826.00000,105255.000000,258626.771977,124.490623,133.554870


In [79]:
external.loc[external["Year_Month"] == "2008-01"]

,"FTR Truck Loadings (000s, SA)_freight_m_trucking",FTR Truck Loadings Index (2000=100)_freight_m_trucking,"FTR Active Truck Utilization (%, SA)_freight_m_trucking","Dry Van Trailer Loadings (000s, SA)_freight_m_trucking","Reefer Trailer Loadings (000s, SA)_freight_m_trucking","Flatbed Trailer Loadings (000s, SA)_freight_m_trucking","Tank Trailer Loadings (000s, SA)_freight_m_trucking","Bulk Trailer/Straight Truck Loadings (000s, SA)_freight_m_trucking","Food & Kindred Products (000s, SA)_freight_m_trucking","Stone, Clay, Glass & Concrete (000s, SA)_freight_m_trucking",...,"East Coast Port Activity, Total TEUs_Exports_y.1","East Coast Port Activity, Total TEUs_SA Imports_y.1","Gulf Coast Port Activity, Total TEUs_Imports_y.1","Gulf Coast Port Activity, Total TEUs_Exports_y.1","Gulf Coast Port Activity, Total TEUs_SA Imports_y.1","Western Canadian Port Activity, Total TEUs_Imports_y.1","Western Canadian Port Activity, Total TEUs_Exports_y.1","Western Canadian Port Activity, Total TEUs_SA Imports_y.1",Total Intermodal (w/o FSC)_y.1,Total Intermodal (w/ FSC)_y.1
189,57633.608984,102.678075,88.29465,18021.824048,3940.638792,6425.307387,6915.782704,22330.056054,7126.929795,5258.820187,...,446657.73,572374.928714,58486.56,90322.59,61444.608268,97597.0,70230.0,92712.867945,99.640213,112.481465


In [61]:
external_updated = external.drop(external.index[0:189])
external_updated = external_updated.reset_index(drop=True)
external_updated

,"FTR Truck Loadings (000s, SA)_freight_m_trucking",FTR Truck Loadings Index (2000=100)_freight_m_trucking,"FTR Active Truck Utilization (%, SA)_freight_m_trucking","Dry Van Trailer Loadings (000s, SA)_freight_m_trucking","Reefer Trailer Loadings (000s, SA)_freight_m_trucking","Flatbed Trailer Loadings (000s, SA)_freight_m_trucking","Tank Trailer Loadings (000s, SA)_freight_m_trucking","Bulk Trailer/Straight Truck Loadings (000s, SA)_freight_m_trucking","Food & Kindred Products (000s, SA)_freight_m_trucking","Stone, Clay, Glass & Concrete (000s, SA)_freight_m_trucking",...,"East Coast Port Activity, Total TEUs_Exports_y.1","East Coast Port Activity, Total TEUs_SA Imports_y.1","Gulf Coast Port Activity, Total TEUs_Imports_y.1","Gulf Coast Port Activity, Total TEUs_Exports_y.1","Gulf Coast Port Activity, Total TEUs_SA Imports_y.1","Western Canadian Port Activity, Total TEUs_Imports_y.1","Western Canadian Port Activity, Total TEUs_Exports_y.1","Western Canadian Port Activity, Total TEUs_SA Imports_y.1",Total Intermodal (w/o FSC)_y.1,Total Intermodal (w/ FSC)_y.1
0,57633.608984,102.678075,88.294650,18021.824048,3940.638792,6425.307387,6915.782704,22330.056054,7126.929795,5258.820187,...,446657.730000,5.723749e+05,58486.560000,90322.590000,61444.608268,97597.00000,70230.000000,92712.867945,99.640213,112.481465
1,57175.659828,101.862208,87.998407,17917.808139,3927.115595,6323.986294,6836.939472,22169.810328,7090.093356,5112.762026,...,472359.530000,5.943458e+05,62079.760000,100410.660000,69356.289323,105664.00000,86567.000000,123918.281279,99.892838,112.865989
2,56814.700553,101.219136,87.790155,17835.922848,3922.853376,6352.228095,6758.129439,21945.566795,7130.846968,5123.243018,...,493039.820000,5.561473e+05,63169.270000,93384.280000,61534.312686,89275.00000,80451.000000,102763.241378,100.466949,113.715206
3,56916.813852,101.401057,89.639867,17713.172439,3978.633479,6304.602493,6858.428982,22061.976458,7139.440571,4996.184950,...,484342.050000,5.689482e+05,63221.100000,101642.940000,63038.826932,107793.00000,84739.000000,108298.068115,102.289780,115.952288
4,56194.246889,100.113756,89.134818,17561.485366,3990.692209,6213.659026,6804.610846,21623.799441,7179.943436,4865.319042,...,499498.010000,5.529787e+05,61657.850000,105424.640000,59864.174184,102278.00000,89548.000000,100455.980122,102.811441,117.038513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,61469.059874,109.511184,89.962431,20072.391023,4379.187334,6993.332683,6571.127779,23453.021054,9375.896057,5404.795100,...,472910.156297,8.175415e+05,131560.759686,125276.157772,124175.287958,225515.00000,103172.000000,203343.596376,119.865470,128.935612
151,62140.797176,110.707928,90.641835,20379.910791,4404.415547,7096.862705,6601.260917,23658.347216,9520.297567,5381.920841,...,499597.784894,9.441444e+05,150026.806042,125319.157772,144616.508568,235159.00000,93979.000000,212554.287841,121.915166,130.511416
152,62170.174317,110.760265,90.587889,20315.722086,4381.188114,7049.924576,6556.745680,23866.593861,9472.590699,5253.495206,...,492325.048120,9.480849e+05,156153.318284,117576.128306,154165.434806,216790.00000,103129.000000,200875.373073,123.509489,131.952815
153,62870.759335,112.008404,94.367218,20404.155544,4436.920215,7199.060314,6523.796564,24306.826698,9565.479621,5479.173341,...,546879.252290,9.980584e+05,173631.303295,123604.184721,162787.138608,260826.00000,105255.000000,258626.771977,124.490623,133.554870


### Only keep the data from 2008 for books and packaging dataset

In [94]:
books_agg.loc[books_agg["Year_Month"]=="2008-01"]

,Year_Month,Sum of Sales
177,2008-01,282604.11


In [95]:
books_updated = books_agg.drop(books_agg.index[0:177])
books_updated = books_updated.reset_index(drop=True)
books_updated.head()

,Year_Month,Sum of Sales
0,2008-01,282604.11
1,2008-02,262474.40
2,2008-03,326059.68
3,2008-04,215387.49
4,2008-05,499559.90


In [96]:
packaging_agg.loc[packaging_agg["Year_Month"] == "2008-01"].head(1)

,Year_Month,Sum of Sales
177,2008-01,381696.51


In [98]:
packaging_updated = packaging_agg.drop(packaging_agg.index[0:177])
packaging_updated = packaging_updated.reset_index(drop=True)
packaging_updated.head()

,Year_Month,Sum of Sales
0,2008-01,381696.51
1,2008-02,430305.85
2,2008-03,451627.81
3,2008-04,467417.48
4,2008-05,462676.37


### Merge packaging and books datasets into external dataset

In [102]:
books_ext = pd.merge(books_updated, external_updated, left_on='Year_Month', right_on='Year_Month', how='left')
pack_ext = pd.merge(packaging_updated, external_updated, left_on='Year_Month', right_on='Year_Month', how='left')

#### books dataset after merged

In [103]:
print(books_ext.shape)
books_ext.head(10)

(157, 641)


,Year_Month,Sum of Sales,"FTR Truck Loadings (000s, SA)_freight_m_trucking",FTR Truck Loadings Index (2000=100)_freight_m_trucking,"FTR Active Truck Utilization (%, SA)_freight_m_trucking","Dry Van Trailer Loadings (000s, SA)_freight_m_trucking","Reefer Trailer Loadings (000s, SA)_freight_m_trucking","Flatbed Trailer Loadings (000s, SA)_freight_m_trucking","Tank Trailer Loadings (000s, SA)_freight_m_trucking","Bulk Trailer/Straight Truck Loadings (000s, SA)_freight_m_trucking",...,"East Coast Port Activity, Total TEUs_Exports_y.1","East Coast Port Activity, Total TEUs_SA Imports_y.1","Gulf Coast Port Activity, Total TEUs_Imports_y.1","Gulf Coast Port Activity, Total TEUs_Exports_y.1","Gulf Coast Port Activity, Total TEUs_SA Imports_y.1","Western Canadian Port Activity, Total TEUs_Imports_y.1","Western Canadian Port Activity, Total TEUs_Exports_y.1","Western Canadian Port Activity, Total TEUs_SA Imports_y.1",Total Intermodal (w/o FSC)_y.1,Total Intermodal (w/ FSC)_y.1
0,2008-01,282604.11,57633.608984,102.678075,88.294650,18021.824048,3940.638792,6425.307387,6915.782704,22330.056054,...,446657.73,572374.928714,58486.56,90322.59,61444.608268,97597.0,70230.0,92712.867945,99.640213,112.481465
1,2008-02,262474.40,57175.659828,101.862208,87.998407,17917.808139,3927.115595,6323.986294,6836.939472,22169.810328,...,472359.53,594345.835988,62079.76,100410.66,69356.289323,105664.0,86567.0,123918.281279,99.892838,112.865989
2,2008-03,326059.68,56814.700553,101.219136,87.790155,17835.922848,3922.853376,6352.228095,6758.129439,21945.566795,...,493039.82,556147.307758,63169.27,93384.28,61534.312686,89275.0,80451.0,102763.241378,100.466949,113.715206
3,2008-04,215387.49,56916.813852,101.401057,89.639867,17713.172439,3978.633479,6304.602493,6858.428982,22061.976458,...,484342.05,568948.228175,63221.10,101642.94,63038.826932,107793.0,84739.0,108298.068115,102.289780,115.952288
4,2008-05,499559.90,56194.246889,100.113756,89.134818,17561.485366,3990.692209,6213.659026,6804.610846,21623.799441,...,499498.01,552978.697408,61657.85,105424.64,59864.174184,102278.0,89548.0,100455.980122,102.811441,117.038513
5,2008-06,576668.11,55726.615571,99.280640,88.925743,17494.055796,3963.799980,6103.120136,6759.024721,21406.614938,...,495633.80,555491.193382,61565.86,102145.83,59591.161856,105416.0,81948.0,106820.337855,102.959165,117.897052
6,2008-07,192745.31,55459.836889,98.805357,90.591021,17381.580858,3997.151230,6086.330847,6650.154837,21344.619116,...,486374.03,552476.408874,64491.43,104947.80,60871.052358,118098.0,78917.0,106487.249384,102.616745,119.369966
7,2008-08,180972.40,54439.248884,96.987112,89.205238,17078.482254,3939.520021,6001.487183,6556.945836,20862.813590,...,485388.15,550351.992581,62769.47,102782.44,60505.864489,112624.0,77031.0,101797.992481,102.103750,119.141590
8,2008-09,206606.68,53302.520846,94.961956,85.336575,16725.527541,4000.945605,5845.469829,6146.165871,20584.412000,...,470167.04,577606.746116,57268.29,76747.74,56539.245694,115593.0,70990.0,107107.278932,101.303973,118.053985
9,2008-10,361073.92,55683.391134,99.203633,89.773489,16557.305879,4062.796519,5869.836122,7517.237463,21676.215149,...,458007.23,563566.214907,72438.38,97921.48,67914.231949,99103.0,68314.0,98267.385089,100.118404,115.627076


In [105]:
books_ext.to_csv("Books_ext.csv")

#### packaging dataset after merged

In [104]:
print(pack_ext.shape)
pack_ext.head()

(157, 641)


,Year_Month,Sum of Sales,"FTR Truck Loadings (000s, SA)_freight_m_trucking",FTR Truck Loadings Index (2000=100)_freight_m_trucking,"FTR Active Truck Utilization (%, SA)_freight_m_trucking","Dry Van Trailer Loadings (000s, SA)_freight_m_trucking","Reefer Trailer Loadings (000s, SA)_freight_m_trucking","Flatbed Trailer Loadings (000s, SA)_freight_m_trucking","Tank Trailer Loadings (000s, SA)_freight_m_trucking","Bulk Trailer/Straight Truck Loadings (000s, SA)_freight_m_trucking",...,"East Coast Port Activity, Total TEUs_Exports_y.1","East Coast Port Activity, Total TEUs_SA Imports_y.1","Gulf Coast Port Activity, Total TEUs_Imports_y.1","Gulf Coast Port Activity, Total TEUs_Exports_y.1","Gulf Coast Port Activity, Total TEUs_SA Imports_y.1","Western Canadian Port Activity, Total TEUs_Imports_y.1","Western Canadian Port Activity, Total TEUs_Exports_y.1","Western Canadian Port Activity, Total TEUs_SA Imports_y.1",Total Intermodal (w/o FSC)_y.1,Total Intermodal (w/ FSC)_y.1
0,2008-01,381696.51,57633.608984,102.678075,88.294650,18021.824048,3940.638792,6425.307387,6915.782704,22330.056054,...,446657.73,572374.928714,58486.56,90322.59,61444.608268,97597.0,70230.0,92712.867945,99.640213,112.481465
1,2008-02,430305.85,57175.659828,101.862208,87.998407,17917.808139,3927.115595,6323.986294,6836.939472,22169.810328,...,472359.53,594345.835988,62079.76,100410.66,69356.289323,105664.0,86567.0,123918.281279,99.892838,112.865989
2,2008-03,451627.81,56814.700553,101.219136,87.790155,17835.922848,3922.853376,6352.228095,6758.129439,21945.566795,...,493039.82,556147.307758,63169.27,93384.28,61534.312686,89275.0,80451.0,102763.241378,100.466949,113.715206
3,2008-04,467417.48,56916.813852,101.401057,89.639867,17713.172439,3978.633479,6304.602493,6858.428982,22061.976458,...,484342.05,568948.228175,63221.10,101642.94,63038.826932,107793.0,84739.0,108298.068115,102.289780,115.952288
4,2008-05,462676.37,56194.246889,100.113756,89.134818,17561.485366,3990.692209,6213.659026,6804.610846,21623.799441,...,499498.01,552978.697408,61657.85,105424.64,59864.174184,102278.0,89548.0,100455.980122,102.811441,117.038513


In [106]:
pack_ext.to_csv("Packaging_ext.csv")